##### Copyright 2020 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=Afd8bu4xJOgh).

In [0]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

# The Mandelbrot set

This notebook provides an example of how to calculate and visualize the Mandelbrot set using Swift and the accelerated Tensor type.

## Attaching to the CPU or an accelerator

In [0]:
import Foundation
import TensorFlow

Let's verify what accelerator, if any, this instance is using. We'll set up a device using the XLA-based X10 backend. ([This tutorial](https://colab.research.google.com/github/tensorflow/swift/blob/main/docs/site/tutorials/introducing_x10.ipynb) has more information on the two backends.)

In [0]:
let device = Device.defaultXLA
// let device = Device.defaultTFEager
print("\u{001B}[2J")

Then, we'll query that device to see what accelerator it found. If none, it will use the CPU.

In [0]:
device

## Configuring complex number tensors

The Mandelbrot set is calculated using complex numbers. To represent a matrix of complex numbers, we'll create a ComplexTensor type that contains separate Tensors for the real and imaginary components.

A precondition that the `real` and `imaginary` tensor shapes match would make sense in practice, but for simplicity we won't verify that here.

In [0]:
struct ComplexTensor {
  let real: Tensor<Float>
  let imaginary: Tensor<Float>
}

We can then define the mathematical operators that we'll need to work over this new parallel complex type.

In [0]:
func +(lhs: ComplexTensor, rhs: ComplexTensor) -> ComplexTensor {
  let real = lhs.real + rhs.real
  let imaginary = lhs.imaginary + rhs.imaginary
  return ComplexTensor(real: real, imaginary: imaginary)
}

func *(lhs: ComplexTensor, rhs: ComplexTensor) -> ComplexTensor {
  let real = lhs.real .* rhs.real - lhs.imaginary .* rhs.imaginary
  let imaginary = lhs.real .* rhs.imaginary + lhs.imaginary .* rhs.real
  return ComplexTensor(real: real, imaginary: imaginary)
}

func abs(_ value: ComplexTensor) -> Tensor<Float> {
  return value.real .* value.real + value.imaginary .* value.imaginary
}

And then test them out:

In [0]:
let complex1 = ComplexTensor(
  real: Tensor<Float>([1.0, 2.0], on: device),
  imaginary: Tensor<Float>([2.0, 3.0], on: device))
let complex2 = ComplexTensor(
  real: Tensor<Float>([2.5, 3.5], on: device),
  imaginary: Tensor<Float>([3.5, 4.5], on: device))
complex1 + complex2

In [0]:
complex1 * complex2

In [0]:
abs(complex1)

## Calculating the Mandelbrot set

We can set up the parameters for the space of complex numbers to calculate over:

In [0]:
let xDensity = 1030
let yDensity = 1030
let tolerance: Float = 4.0
let realMinimum: Float = -2.0
let realMaximum: Float = 1.0
let imaginaryMinimum: Float = -1.3
let imaginaryMaximum: Float = 1.3
let iterations: Int32 = 200

From the above parameters, ComplexTensors can be initialized that span the complex number space. We also need to configure the variables used in the calculation.

In [0]:
let xs = Tensor<Float>(linearSpaceFrom: realMinimum, to: realMaximum, count: xDensity, on: device)
  .broadcasted(to: [xDensity, yDensity])
let ys = Tensor<Float>(linearSpaceFrom: imaginaryMinimum, to: imaginaryMaximum, count: yDensity, on: device)
  .expandingShape(at: 1).broadcasted(to: [xDensity, yDensity])
let X = ComplexTensor(real: xs, imaginary: ys)
var Z = ComplexTensor(real: Tensor(zerosLike: xs), imaginary: Tensor(zerosLike: ys))
var divergence = Tensor<Float>(repeating: Float(iterations), shape: xs.shape, on: device)

// For X10, we'll make sure the initialization of these tensors doesn't carry
// into the trace for the first iteration.
LazyTensorBarrier()

Finally, we can perform the actual Mandelbrot set calculation for a specified number of iterations. We mark the iteration at which the calculation diverges, if it does, for later visualization.

In [0]:
let start = Date()

for iteration in 0..<iterations {
  Z = Z * Z + X

  let aboveThreshold = abs(Z) .> tolerance
  divergence = divergence.replacing(with: min(divergence, Float(iteration)), where: aboveThreshold)

  // For X10, we're cutting the trace to be a single iteration.
  LazyTensorBarrier()
}

print("Total calculation time: \(String(format: "%.3f", Date().timeIntervalSince(start))) seconds")


## Visualizing the Mandelbrot set

At each point in the complex space we operated over, `divergence` now contains the iteration at which the calculation diverged for that starting complex number. If it did not, it contains the maximum number of iterations.

We'll pull in Matplotlib to help us visualize the results inline:

In [0]:
import PythonKit
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")
let plt = Python.import("matplotlib.pyplot")


set up a function to plot the set over a given area:

In [0]:
func plotMandelbrot(_ divergence: Tensor<Float>, xLimits: (Float, Float), yLimits: (Float, Float)) {
  let fig = plt.figure(figsize: [10, 10])

  let ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon: false, aspect: 1)
  ax.set_xticks(Array<Int>([]))
  ax.set_yticks(Array<Int>([]))
  ax.set_xlim(xLimits.0, xLimits.1)
  ax.set_ylim(yLimits.0, yLimits.1)

  let colorMap = plt.get_cmap("prism")
  colorMap.set_over("black")

  plt.imshow(divergence.scalars.makeNumpyArray().reshape([xDensity, yDensity]),
    extent: [realMinimum, realMaximum, imaginaryMinimum, imaginaryMaximum],
    cmap: colorMap, vmax: (iterations - 1))

  plt.show()  
}

and then display the calculated Mandelbrot set at a few magnifications:

In [0]:
plotMandelbrot(divergence, xLimits: (realMinimum, realMaximum), yLimits: (imaginaryMinimum, imaginaryMaximum))

In [0]:
plotMandelbrot(divergence, xLimits: (-1.0, 0.0), yLimits: (0.0, 1.0))

In [0]:
plotMandelbrot(divergence, xLimits: (-1.0, -0.5), yLimits: (0.0, 0.4))